___

<a href='https://www.udemy.com/user/joseportilla/'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Content Copyright by Pierian Data</em></center>

# Milestone Project 2 - Blackjack Game
In this milestone project you will be creating a Complete BlackJack Card Game in Python.

Here are the requirements:

* You need to create a simple text-based [BlackJack](https://en.wikipedia.org/wiki/Blackjack) game
* The game needs to have one player versus an automated dealer.
* The player can stand or hit.
* The player must be able to pick their betting amount.
* You need to keep track of the player's total money.
* You need to alert the player of wins, losses, or busts, etc...

And most importantly:

* **You must use OOP and classes in some portion of your game. You can not just use functions in your game. Use classes to help you define the Deck and the Player's hand. There are many right ways to do this, so explore it well!**


Feel free to expand this game. Try including multiple players. Try adding in Double-Down and card splits! Remember to you are free to use any resources you want and as always:

# HAVE FUN!

In [1]:
import random
from IPython.display import clear_output

# Define the standard 52-card deck of playing cards
suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Ace':1,
          'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10,
          'Jack':10, 'Queen':10, 'King':10}

In [2]:
class Card:
    def __init__(self,suit,rank):
        self.suit = suit
        self.rank = rank
        self.value = values[rank]

    def __str__(self):
        return self.rank + ' of ' + self.suit

In [3]:
class Deck:
    def __init__(self, pack_amount = 1):
        self.all_cards = []
        for i in range(pack_amount):
            for suit in suits:
                for rank in ranks:
                    self.all_cards.append(Card(suit,rank))

        random.shuffle(self.all_cards)

    def deal_one(self):
        # Note that pop() removes one card from the list
        return self.all_cards.pop()

    def __len__(self):
        return len(self.all_cards)

In [4]:
class Player:
    def __init__(self, name, balance):
        self.name = name
        self.balance = int(balance)
        # Each player starts without cards
        self.new_round()

    def new_round(self):
        self.all_cards = []
        self.value = 0

    def add_card(self, new_card):
        self.all_cards.append(new_card)
        self.value += new_card.value

    def get_bet_amount(self):
        self.bet = input(player_one.name + ', place bet (E'+str(player_one.balance)+' balance) for this Blackjack round: E')
        self.bet = int(self.bet)
        return self.bet
    
    def bet_lost(self):
        self.balance -= self.bet

    def bet_won(self):
        self.balance += self.bet * 2

In [5]:
class Dealer(Player):
    def __init__(self, pack_amount = 1):
        self.new_round()
        self.standing = False

        # Used for is_hitting_solution2()
        # Define variables for probability calculation, based on amount of cards in the deck
        self.cards_value_one_to_nine = 4 * pack_amount
        #self.cards_value_ten = 4 * 4 * pack_amount
        self.cards_amount = 52 * pack_amount

    def is_hitting(self, debug = False):
        #return self.is_hitting_solution1(debug)
        return self.is_hitting_solution2(debug)

    def is_hitting_solution1(self, debug = False):
        # Solution based on leftover value and highest card
        highest_card = max(values.values())
        if self.value <= (21 - highest_card):
            return True
        rand = random.random()
        if (21 - self.value) / highest_card > rand:
            if debug:
                print('...debug: change', (21 - self.value) / highest_card, '>', rand)
            return True
        
        self.standing = True
        return False
    
    def is_hitting_solution2(self, debug = False):
        # Solution based on the amount of cards in the deck(s)
        highest_card = max(values.values())
        score_leftover = 21 - self.value

        probability = 1.0
        if score_leftover <= 9:
            probability = score_leftover * self.cards_value_one_to_nine / self.cards_amount
        rand = random.random()
        if debug:
            print('...debug: change', probability, rand)
        if probability > rand:
            return True
        
        self.standing = True
        return False

In [6]:
def show_score():
    print('SCORES |', 'Dealer:', dealer.value, '|', player_one.name + ':', player_one.value)

def is_loser(player):
    if player.value > 21:
        return True
    return False

def is_winner(player):
    if player.value == 21:
        return True
    if player.value > dealer.value:
        return True
    return False

In [ ]:
# Store (multiple) decks in the dealing shoe
dealing_shoe = Deck(pack_amount=2)
player_one = Player(name='Player One', balance=100)
dealer = Dealer(pack_amount=2)

# Start the game
game_on = True
while game_on:
    # New round, put cards back in shoe and reset variables
    dealing_shoe.all_cards.extend(player_one.all_cards)
    dealing_shoe.all_cards.extend(dealer.all_cards)
    player_one.new_round()
    dealer.new_round()
    last_round = False

    # Ask player for betting amount
    player_one.get_bet_amount()

    # Play blackjack round
    blackjack_on = True
    while blackjack_on:
        # Determine dealer choice
        if dealer.is_hitting(debug=True):
            # Draw a card for the dealer
            new_card = dealing_shoe.deal_one()
            dealer.add_card(new_card)
            print('Dealer: hitting', new_card)
        else:
            print('Dealer: standing', new_card)

        # Dealer is over 21? Player wins, update player bet, and early stop this round
        if dealer.value > 21:
            print(player_one.name + ': winner of round, dealer hitting over 21 (', dealer.value,')')
            player_one.bet_won()
            blackjack_on = False
            break

        # Show score for player to base choise on
        show_score()

        if last_round == False:
            # Ask player to hit or stand
            next_move = input(player_one.name + ': (h)it or (s)tand?')
            if next_move == 'h':
                # Draw a card for the player
                new_card = dealing_shoe.deal_one()
                player_one.add_card(new_card)
                print(player_one.name + ': hitting', new_card)
            elif next_move == 's':
                last_round = True
                print(player_one.name + ': standing')

            # Show score of the players
            show_score()

        # Check if player is winner or loser
        if is_loser(player_one):
            player_one.bet_lost()
            blackjack_on = False
            print(player_one.name + ': lost round, hitting over 21')
        elif is_winner(player_one) and False:
            player_one.bet_won()
            blackjack_on = False
            print(player_one.name + ': winner of round')

        # Optional quit option received from player
        if next_move == 'q':
            blackjack_on = False
            game_on = False
            print('quiting the blackjack game')
        # Check if player is without balance, lost the game
        if player_one.balance <= 0:
            blackjack_on = False
            game_on = False
            print(player_one.name + ': lost game, balance reached E 0')